1. Create a heat map that displays the humidity for every city from the part I of the homework.

2. Narrow down the DataFrame to find your ideal weather condition. For example: (A max temperature lower than 80 degrees but higher than 70., Wind speed less than 10 mph., Zero cloudiness.) Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal. Note: Feel free to adjust to your specifications but be sure to limit the number of rows returned by your API requests to a reasonable number.

3. Using Google Places API to find the first hotel for each city located within 5000 meters of your coordinates.

4. Plot the hotels on top of the humidity heatmap with each pin containing the Hotel Name, City, and Country.

In [177]:
#Import dependencies
from config import gkey
import gmaps
import numpy as np
import pandas as pd
import requests

In [178]:
#Import csv
city_data = "Resources/citydata.csv"

# Read City Data File and store into Pandas DataFrames
city_data = pd.read_csv(city_data)

In [179]:
# Configure gmaps with API key
gmaps.configure(api_key=gkey)

In [180]:
# Store 'Lat' and 'Lng' into  locations 
locations = city_data[["Latitude", "Longitude"]].astype(float)

# Convert Poverty Rate to float and store
humidity = city_data["Humidity"].astype(float)

In [181]:
# Create a poverty Heatmap layer
fig = gmaps.figure(center=(0,0), zoom_level=2)

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 2)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [274]:
#Narrow down data frame
# Temp between 60-75, Wind speed less than 10, Clouds less than 20, Humidity less than 55
ideal_subset=city_data.loc[(city_data['Temperature'] >= 60) & (city_data['Temperature'] <= 72) & 
                           (city_data['Wind Speed'] <= 10) & (city_data['Cloudiness'] <= 0) & (city_data['Humidity'] < 55)]

#Drop unwanted columns
ideal_subset = ideal_subset.drop(columns=['Temperature', 'Humidity', 'Cloudiness', 'Wind Speed', 'Date'])

# set up additional columns to hold information
ideal_subset['Hotel Name'] = ""
ideal_subset

,City Name,Latitude,Longitude,Country,Hotel Name
155,bulawayo,-20.15,28.58,ZW,
258,turayf,31.67,38.66,SA,
314,nizwa,22.93,57.53,OM,
317,katra,32.98,74.95,IN,
515,abha,18.22,42.51,SA,
535,faya,18.39,42.45,SA,
590,hun,29.13,15.95,LY,
593,buraydah,26.33,43.98,SA,
646,marzuq,14.40,46.47,YE,


In [275]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "radius": 5000,
    "type": "lodging",
    "key": gkey,
}
# use iterrows to iterate through pandas dataframe
for index, row in ideal_subset.iterrows():
    # get lat, lng from df
    lat = row[1]
    lng = row[2]

    # change location each iteration while leaving original params in place
    params["location"] = (f"{lat},{lng}")

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}.")

    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        
        ideal_subset.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

Retrieving Results for Index 155.
Closest hotel is 4 On Housman Backpackers.
Retrieving Results for Index 258.
Closest hotel is Swiss Spirit Hotel & Suites Turaif.
Retrieving Results for Index 314.
Closest hotel is Orient Apartments.
Retrieving Results for Index 317.
Closest hotel is KC Residency Vaishnodevi.
Retrieving Results for Index 515.
Closest hotel is Abha Hotel.
Retrieving Results for Index 535.
Closest hotel is مقبرة ال مجمل.
Retrieving Results for Index 590.
Closest hotel is Alrwasi Hotel.
Retrieving Results for Index 593.
Closest hotel is Mövenpick Hotel Qassim.
Retrieving Results for Index 646.
Missing field/result... skipping.


In [276]:
#Drop empty row
hotel_df=ideal_subset[:-1]

In [277]:
hotel_df

,City Name,Latitude,Longitude,Country,Hotel Name
155,bulawayo,-20.15,28.58,ZW,4 On Housman Backpackers
258,turayf,31.67,38.66,SA,Swiss Spirit Hotel & Suites Turaif
314,nizwa,22.93,57.53,OM,Orient Apartments
317,katra,32.98,74.95,IN,KC Residency Vaishnodevi
515,abha,18.22,42.51,SA,Abha Hotel
535,faya,18.39,42.45,SA,مقبرة ال مجمل
590,hun,29.13,15.95,LY,Alrwasi Hotel
593,buraydah,26.33,43.98,SA,Mövenpick Hotel Qassim


In [278]:
#Store locations
hotel_locations = hotel_df[["Latitude", "Longitude"]].astype(float)

In [295]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City Name}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [296]:
# Add marker layer and info box content ontop of heat map
hotel_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add the layer to the map
fig.add_layer(hotel_layer)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))

In [297]:
# Create a combined map with hotels and heat map
fig = gmaps.figure(center=(0,0), zoom_level=2)

fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)

fig

Figure(layout=FigureLayout(height='420px'))